## AlekseevDP(DSU-4,DLL-15)_HW#13_Работа с текстом(2)_Исправленное

## Задание: реализовать задачу классификации на основе BERT-like модели (п.1) и KNN (п.2) на данных "Russian Intents Dataset": https://www.kaggle.com/datasets/constantinwerner/qa-intents-dataset-university-domain

## **Уточненное задание (по результатам проверки ДЗ преподавателем): 3. Обучение KNN на эмбеддингах из Bert-подобной сети, с использованием одной из библиотек nmslib/faiss/scann (соединение токенизации с берта и классификации с KNN).**

## 1. Классификация на основе BERT-like модели (используем pretrained-модель 'bert-base-multilingual-cased' с ресурса Huggingface)

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.0 MB/s eta 0:00:00


## Загрузим и проанализируем данные

In [2]:
import pandas as pd
import numpy as np
from transformers import TrainingArguments, Trainer

In [3]:
train=pd.read_csv('/content/dataset_train.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
test =pd.read_csv('/content/dataset_test.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
data = result = pd.concat([train,test])
NUM_LABELS=len(set(data['intent']))
labels=set(data['intent'])
id2label = {i:label for i,label in enumerate(labels) if label != None}
label2id = {label:i for i,label in enumerate(labels) if label != None}
print(data['intent'].value_counts())
print('examples:\n',data['text'].iloc[2]+'\n'+data['text'].iloc[929]+'\n'+data['text'].iloc[24]+'\n'+data['text'].iloc[1224])

sched_teacher                1184
sched_for_group               433
sched_for_group_day           429
wifi                          306
status_free                   288
                             ... 
trade_union_general            24
student_trade_union_enter      23
nsu_foundation_date            23
location_general               21
staff_trade_union_enter        21
Name: intent, Length: 142, dtype: int64
examples:
 взять справку
где столовка в новом здании
мне нужна справка ааа
деканат фф расположен где


## Загрузим модель и токенизатор

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = 'bert-base-multilingual-cased'
N_MODELS = 3
tokenizers = []
models = []
for i in range(N_MODELS):
    tokenizers.append(AutoTokenizer.from_pretrained(MODEL_NAME))
    models.append(AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,label2id=label2id,id2label=id2label,num_labels =NUM_LABELS))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

## Подготовим трейн- и тест-датасеты

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def make_dataset(df):
    labels = [label2id[label] for label in df['intent']]
    texts = [tokenizer(text,padding='max_length', max_length = 32, truncation=True) for text in df['text']]
    
    result = []
    for idx,(text,label) in enumerate(zip(texts,labels)):
        result.append({"input_ids" : text['input_ids'], 'attention_mask' : text['attention_mask'], "label" : label})
    return result

train_dataset = make_dataset(train)
test_dataset = make_dataset(test)

In [6]:
train_dataset[0]

{'input_ids': [101,
  67251,
  554,
  10227,
  26771,
  64962,
  10521,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'label': 29}

## Обучим модель

In [7]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [9]:
for model in models:
    args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.05,
    load_best_model_at_end=False,
    report_to=None,
    metric_for_best_model = 'accuracy'
    )
    trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )
    trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13230
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2070
  Number of trainable parameters = 177962638
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` met

Step,Training Loss,Validation Loss,Accuracy
500,2.954400,1.437221,0.841450
1000,1.032500,0.557379,0.936580
1500,0.486100,0.322277,0.951302
2000,0.337400,0.262931,0.951302


***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-500
Configuration saved in test-glue/checkpoint-500/config.json
Model weights saved in test-glue/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1000
Configuration saved in test-glue/checkpoint-1000/config.json
Model weights saved in test-glue/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1500
Configuration saved in test-glue/checkpoint-1500/config.json
Mod

Step,Training Loss,Validation Loss,Accuracy
500,2.991900,1.511102,0.800680
1000,1.090200,0.598186,0.936580
1500,0.514000,0.341771,0.954700
2000,0.353400,0.277404,0.953567


***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-500
Configuration saved in test-glue/checkpoint-500/config.json
Model weights saved in test-glue/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1000
Configuration saved in test-glue/checkpoint-1000/config.json
Model weights saved in test-glue/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1500
Configuration saved in test-glue/checkpoint-1500/config.json
Mod

Step,Training Loss,Validation Loss,Accuracy
500,2.950900,1.461200,0.795017
1000,1.043900,0.576837,0.930917
1500,0.496000,0.336039,0.949037
2000,0.342100,0.276656,0.950170


***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-500
Configuration saved in test-glue/checkpoint-500/config.json
Model weights saved in test-glue/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1000
Configuration saved in test-glue/checkpoint-1000/config.json
Model weights saved in test-glue/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 883
  Batch size = 32
Saving model checkpoint to test-glue/checkpoint-1500
Configuration saved in test-glue/checkpoint-1500/config.json
Mod

## Проведем непосредственно классификацию и ансамблирование

In [10]:
class ClassifierLanguageModel():
	def __init__(self, model,tokenizer):
		self.model = model
		self.tokenizer = tokenizer

	def run(self, text):
		model_input = self.tokenizer.encode(text, return_tensors='pt').cuda()
		model_output = self.model.bert.config.id2label[self.model(model_input)['logits'].argmax().item()]
		return model_output

class Ensemble():
    def __init__(self,predictors):
        self.predictors = predictors
            
    def run(self,string):
        predictions = []
        for predictor in self.predictors:
            predictions.append(predictor.run(string))
        counter = {}
        for p_i in predictions:
            if p_i not in counter.keys():
                counter[p_i] = 0
            counter[p_i] += 1
        
        # check what is the majority have voted for
        for key in counter.keys():
            if counter[key] > 0.5*len(predictions):
                return key
            
        return 'out_of_class'

In [11]:
predictors = []
for model, tokenizer in zip(models,tokenizers):
    predictors.append(ClassifierLanguageModel(model,tokenizer))
ensemble = Ensemble(predictors)

## Определим качество классификации с использованием f1-метрики

In [12]:
reference = []
predictions = []
for text,label in zip(test['text'],test['intent']):
    output = ensemble.run(text)
    predictions.append(output)
    reference.append(label)

In [13]:
reference[:10]

['statement_general',
 'statement_general',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform']

In [14]:
predictions[:10]

['statement_general',
 'statement_general',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform',
 'conform']

In [15]:
from sklearn.metrics import f1_score

for average_type in ['macro','micro','weighted']:
    print(f'{average_type} F1 = {f1_score(reference, predictions, average=average_type)}')

macro F1 = 0.9274263673207336
micro F1 = 0.9501698754246886
weighted F1 = 0.9505289691643032


## Удостоверимся, что классификатор способен "отсеивать" out-of-classes запросы пользователей (т.е. не подходящие ни под одну из меток классификатора)

In [16]:
test = [['где можно погулять в Академгородке?','loc_walk']\
        ,['где недорого поесть на 3 этаже нового корпуса?','loc_cafeteria_new_building_3_etage'],\
        ['как поменять колесо с помощью домкрата?','out_of_classes']] # offtopic question

for phrase in test:
    label = ensemble.run(phrase[0])
    print(label,phrase[1])

loc_walk loc_walk
loc_cafeteria_new_building_3_etage loc_cafeteria_new_building_3_etage
loc_eat out_of_classes


## Мы видим, что качество предсказаний с использованием предобученной BERT-like модели 'bert-base-multilingual-cased' находится на высоком уровне (f1-метрика = 95%), а также классификатор способен отсеивать out-of-classes запросы пользователей.

## 2. Для классификации методом KNN используем меру TF-IDF

In [17]:
import re
import numpy
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [19]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## Определим функцию предобработки текста (удаление чисел, знаков пунктуации, заглавных букв, а также лемматизация и стемминг)

In [20]:
def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^А-яа-я]", " ", text)
    text = text.lower()

    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('russian')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

## Определим функцию для определения меры TF-IDF для корпуса текста

In [21]:
from nltk.stem.snowball import stopwords
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('russian'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

## Загрузим текст (text) с метками (intent)

In [22]:
train=pd.read_csv('/content/dataset_train.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
test =pd.read_csv('/content/dataset_test.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
all_data = result = pd.concat([train,test])

## Разобьем выборку на трейн- и тест-:
- X - собственно классифицируемый текст: all_data['text'],

- y - целевая переменная (признак класса): all_data['intent']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(all_data['text'], all_data['intent'],
                                                        test_size=0.2, random_state=5)

In [24]:
X_train

3431     как можно получить бесплатно печать атрибутки ...
10310                                          попасть атм
10817            находится кафе корпус главный этаж второй
9042     разрешение на вход для гостей мероприятия полу...
12222                            где сейчас Тимофеева О.А.
                               ...                        
3046                               стол военный обнаружить
9917                                   когда экз находится
4079               распологается где хор академический нгу
2254                          ехать институт электрометрии
2915                      как пройти новый корпус подскажи
Name: text, Length: 11290, dtype: object

## Создадим меры TF-IDF для классифицируемого текста (X_train, X_test)

In [26]:
X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=False, lemmatize=True, stemmer=False)

## Создадим и обучим классификатор KNN

In [32]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

In [33]:
knn.fit(X_train, y_train)
predicted = knn.predict(X_test)

In [34]:
predicted

array(['loc_torgovy_center', 'sched_general', 'loc_dean_mmf', ..., 'wifi',
       'loc_atm', 'sched_teacher'], dtype=object)

## Посчитаем метрики качества классификации

In [35]:
acc = metrics.accuracy_score(y_test, predicted)
print('KNN with TF-IDF accuracy = ' + str(round(acc*100,3)) + '%')

KNN with TF-IDF accuracy = 85.158%


In [36]:
scores = cross_val_score(knn, X_train, y_train, cv=3)
print("Cross Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)

Cross Validation Accuracy: 0.83 (+/- 0.01)
[0.82678002 0.83709806 0.83284613]


## Можно сделать вывод, что качество классификации текста ("Russian Intents Dataset") методом KNN с использованием меры TF-IDF получилось на достаточно высоком уровне (KNN with TF-IDF accuracy = 85%), но хуже, чем с использованием BERT-like модели (f1 score = 95%).

## **3. Обучение KNN на эмбеддингах из Bert-подобной сети, с использованием одной из библиотек nmslib/faiss/scann (соединение токенизации с берта и классификации с KNN).**

In [1]:
import torch

In [56]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min = 1e-9)

In [3]:
from transformers import AutoTokenizer, AutoModel

In [39]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
train = pd.read_csv('d:/dataset_train.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])
test = pd.read_csv('d:/dataset_test.tsv',delimiter='\t',encoding="utf-8",names=['text', 'intent'])

In [41]:
train_tokens = tokenizer(train['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
test_tokens = tokenizer(test['text'].tolist(), padding=True, truncation=True, return_tensors='pt') 

In [42]:
train_tokens[1]

Encoding(num_tokens=39, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [44]:
with torch.no_grad():
    train_embeddings = mean_pooling(model(**train_tokens),train_tokens['attention_mask'])
    test_embeddings = mean_pooling(model(**test_tokens),test_tokens['attention_mask'])

In [14]:
#conda install -c conda-forge nmslib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - nmslib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.11.1              |   py39hcbf5309_1         908 KB  conda-forge
    nmslib-2.1.1               |   py39hefe7e4c_1         480 KB  conda-forge
    pybind11-2.10.1            |   py39h59b6b97_0         236 KB
    pybind11-global-2.10.1     |   py39h59b6b97_0         169 KB
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ruamel.yaml-0.17.21        |   py39hb82d6ee_1         169 KB  conda-forge
    ruamel.yaml.clib-0.2.6     |   py39h2bbff1b_1         101 KB
    ------------------------------------------------------------
                                           Total:         2.0 MB

The following NEW packages will be INSTALLED:

  nmslib            



==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [17]:
import nmslib

In [51]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(train_embeddings)
index.createIndex({'post': 2}, print_progress=True)

In [54]:
for n in np.random.randint(0,test.shape[0],10):
    id,dist=index.knnQuery(test_embeddings[n],k=5)
    print(n, test.iloc[n].text,test.iloc[n].intent)
    for i,d in zip(id,dist):
        print(d,train.iloc[i].text,train.iloc[i].intent)
    print()

316 находится студент ггф сколько stat_numb_of_students
0.065917075 находится студент ммф сколько stat_numb_of_students
0.08664274 находится студент фф сколько stat_numb_of_students
0.09404367 находится студент эф сколько stat_numb_of_students
0.09618044 находится студент сколько stat_numb_of_students
0.10664189 находится студент фен сколько stat_numb_of_students

351 ну что как дела smalltalk_talk
0.11946112 ну что что да как smalltalk_talk
0.15970016 чё как как дела smalltalk_talk
0.2064687 ну что чем занят smalltalk_talk
0.20847398 эй как дела smalltalk_talk
0.22090566 чё как что да как smalltalk_talk

738 корпус административный 4 этаж кафетерий распологается где loc_cafeteria_new_building_4_etage
0.048048973 корпус главный этаж первый кафетерий распологается где loc_cafeteria_new_building_1_etage
0.06231916 корпус новый этаж третий кафетерий распологается где loc_cafeteria_new_building_3_etage
0.07600796 корпус административный этаж первый бистро распологается где loc_cafeteria_ne